In [ ]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2


In [ ]:
import os 
import sys
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import osmnx as ox
%matplotlib qt

from buildings import *
from dem2stl import *

In [ ]:
sys.path.append('..\\numpy2stl')
import numpy2stl as np2stl
from osm2stl import *

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
%matplotlib qt

# Buildings and Cities: Philadelphia 

In [ ]:
NSEW = np.array([39.967, 39.945,-75.135,-75.1925])
GEO_poly = ox.utils_geo.bbox_to_poly(*NSEW)
gdf = building_polygons(GEO_poly)

In [ ]:
poly = ox.geocode_to_gdf('Antwerp,Belgium')
poly2 = ox.project_gdf(poly)
GEO_poly = poly2.loc[0,"geometry"]

In [ ]:
poly 

# Buildings and Cities: Antwerpen

In [ ]:
from shapely.ops import cascaded_union
from shapely.geometry import Polygon,MultiPolygon
from shapely.geometry import polygon

def polygon_to_perimeter(poly):
    
    poly = polygon.orient(poly)
    
    verts,peri = [],[]
    n_v = 0
    exter = np.array(poly.exterior.coords)
    exter = exter[:-1]
    verts.extend(exter)
    peri.append( np.arange(len(exter) + n_v ))
    n_v = len(exter) + n_v 
    
    
    inter = poly.interiors
    for p in inter:
        pts = p.coords[:-1]
        verts.extend( pts )
        peri.append( np.arange(len(pts)) + n_v )
        n_v = len(pts) + n_v             
               
    verts = np.array(verts)
    
    perimeters = []
    for line_idx in peri:
        line = verts[line_idx]
        
        angles = get_perimeter_angles( line) 
        simpified_line = np.array(line_idx[  (angles < 179) | (angles > 181) ])
        perimeters.append(simpified_line)
    

    return verts,perimeters

def polygon_to_prism(polygons,heights,base_val=0):
    all_triangles = []

    for n,poly in enumerate(polygons):
        print(n)
        #if poly.area < 500: continue        
        
        verts, peri = polygon_to_perimeter(poly)
        verts = np.concatenate((verts, verts[:,0:1]*0),axis=1)
        
        verts[:,2] = heights[n]
        try:
            _, faces = np2stl.simplify_surface(verts, peri)
        except: 
            continue
        
        #    print(verts)
        ## Add Z value
        top_tris = verts[faces]
        all_triangles.append( top_tris )
        wall_tris = np2stl.perimeter_to_walls(verts, peri, floor_val=base_val)
        all_triangles.append( wall_tris )

    return all_triangles

def shapely_to_buildings(shp_poly, z0=1,z1=39,polygons=None):
    if polygons is None:
        polygons = []
        
    for poly in shp_poly:
        p = {}
        p['roof_height'] = z1
        p['base_height'] = z0
        p['points'] = np.array(poly.exterior.coords).T
        polygons.append(p)
        
    return polygons

def triangulate_buildings(polygons):

    triangles = []

    for _,p in enumerate( polygons ):

        roof = p['roof_height'] 
        base = p['base_height'] 
        vert = p['points'].T

        #if (np.isclose(vert[0],vert[-1])):   
        vert = vert[:-1]

        zdim = np.zeros((len(vert),1)) + roof
        vert = np.concatenate([vert, zdim],axis=1)
        
        tri = np2stl.polygon_to_prism(vert, base_val=base)
        triangles.append( tri )

    triangles = np.concatenate(triangles)   
    return triangles

def boundry_to_poly(GEO_poly):
    pts = np.array(GEO_poly.exterior.coords).T
    p = {"points":pts,"roof_height":0,"base_height":-30}
    polygons = [p]

    return polygons

def get_waterways( GEO ):
    
    ftpt = ox.footprints_from_polygon(GEO, footprint_type="natural")    
    
    x = ftpt[ftpt["natural"]=="water"]
    x = x.dropna(axis=1, how='all')
    x = x[["geometry","name","waterway","natural"]]
    areas = [i["geometry"].area*10000000 for n,i in x.iterrows()]
    x["areas"] = areas
    x = x[x["areas"]>1]
    
    polys = [ i["geometry"].intersection(GEO) for n,i in x.iterrows()]
    x["geometry"] = polys
    x = ox.project_gdf(x)
    return x 

In [ ]:
ox

In [ ]:
#bounds,bbox = get_boundries_osmnx('Antwerp,Belgium')
NSEW = np.array([51.242, 51.202,  4.42,  4.36])
GEO_poly = ox.utils_geo.bbox_to_poly(*NSEW)

In [ ]:
print("Adding Buildings")
#gdf = building_to_gdf(GEO_poly)
gdf = ox.project_gdf(gdf)
print("Making Polygons")
#building_poly = get_polygons(gdf)
heights = building_heights(gdf)

In [ ]:
building_poly = get_polygons(gdf)
building_poly = [Polygon(poly["points"].T) for poly in building_poly]
building_poly = MultiPolygon(building_poly)
building_poly = cascaded_union(building_poly)
print(building_poly)

In [ ]:
building_poly = shapely_to_buildings(building_poly , z0=0,z1=40)
print(building_poly)

In [ ]:


tris = triangulate_buildings(building_poly)
vertices, faces = np2stl.vertices_to_index(tris)
mesh = trimesh.Trimesh(vertices,faces)
#mesh.export("Antwerpen_buildings.stl")

In [ ]:
mesh.show()

## Get Waterways

In [ ]:
gdf_rivers = get_waterways( GEO_poly )

In [ ]:
polygons = get_polygons(gdf_rivers)

water_poly = [Polygon(poly["points"].T) for poly in polygons]
water_poly = MultiPolygon(water_poly)
water_poly = cascaded_union(water_poly)


In [ ]:
patches = [PolygonPatch(poly) for poly in water_poly]
patches.extend([PolygonPatch(poly) for poly in water_poly])
draw_patches(patches)

In [ ]:
pts = np.array([np.array(water_poly.bounds)[[0,0,2,2,0]],np.array(water_poly.bounds)[[1,3,3,1,1]]])
p = {"points":pts,"roof_height":300,"base_height":1}
base_dict = [p]

In [ ]:
water_dict = shapely_to_buildings(water_poly, z0=-10,z1=1,polygons=None)

In [ ]:
building_poly

In [ ]:

tris = triangulate_buildings(water_dict)
vertices, faces = np2stl.vertices_to_index(tris)
river_mesh = trimesh.Trimesh(vertices,faces)
#river_mesh.export("Antwerpen_Rivers.stl")

## Almanjano

In [ ]:
#bounds,bbox = get_boundries_osmnx('Antwerp,Belgium')
NSEW = np.array([41.8537, 41.848, -2.3345, -2.3421])
GEO_poly = ox.utils_geo.bbox_to_poly(*NSEW)

## Test

In [ ]:
import numpy as np
m, heights = np.load("local.npy", allow_pickle=True)

In [ ]:
np.save("local.npy", np.array([m,heights]))

In [ ]:
tris = polygon_to_prism(m[150:155],heights,base_val=0)

In [ ]:
np.concatenate(tris).shape

In [ ]:
tris

In [ ]:
patches = [PolygonPatch(poly) for poly in m]
draw_patches(patches,heights)

In [ ]:
from skimage.measure import approximate_polygon

In [ ]:
for p in m[150:152]:

    exter = np.array(p.exterior.coords)
    plt.plot(exter[:,0],exter[:,1],"-o")

    #inter = np.array(p.interiors)

In [ ]:
sqr = np.array([[0,0],[1,0],[1,1],[0,1]])
poly = [sqr * 20 + 30, sqr*10 + 100, sqr] 
heights = [10,30,10]

m = [Polygon(p) for p in poly]
m = MultiPolygon(m)
m = cascaded_union(m)

In [ ]:
for t in np.concatenate(tris[:100]):
    plt.plot(t[:,0],t[:,1])
    plt.plot(t[[-1,0],0],t[[-1,0],1])

In [ ]:
tris